### Импорты

In [62]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [118]:
import requests
from pprint import pprint
import time
from bs4 import BeautifulSoup
import re
from collections import defaultdict as dd
from copy import copy
from tqdm import tqdm

In [64]:
session = requests.session()

In [65]:
from fake_useragent import UserAgent
ua = UserAgent(verify_ssl=False)
headers = {'User-Agent': ua.random}
#print(headers)

In [66]:
import random
import json
import pandas as pd
from collections import Counter

In [67]:
import string
from numpy.random import choice
import nltk

In [68]:
# форма поискового запроса к корпусу
req1 = 'http://web-corpora.net/RLC/search/?exact_word=&wordform%5B%5D=&lex%5B%5D=&grammar%5B%5D=&errors%5B%5D='
req2 = '&from%5B%5D=1&to%5B%5D=1&wordform%5B%5D=&lex%5B%5D=&grammar%5B%5D=&errors%5B%5D=&lex_search=Search&date1=&date2=&gender=any&mode=any&background=any&language%5B%5D=fin&format=full&per_page=50&expand=%2B-1&page='

# откуда брать информацию об ошибке
search = 'http://web-corpora.net/RLC/document-annotations/search?document='

In [8]:
# функция, создающая запрос по типу ошибки (для нескольких ищет вхождения с хотя бы одной из списка)
def create_req(error):
    if type(error) == str:
        return req1 + error + req2
    elif type(error) == list:
        return req1 +'('+'%7C'.join(error) + ')' + req2

# Генерация ошибок

## Орфографические

1. Замена буквы
2. Убирание буквы из кластера согласных
3. Мягкий знак
4. Дефисное/пробельное/слитное написание
5. Редукция и гиперкоррекция

### 1. Замена буквы 

In [69]:
# (Данные по заменам вычислены в RLC_crawlers.ipynb)

with open('Counter_pairs.json', 'r', encoding='utf-8') as f:
    pairs = json.load(f)

repl_dict = dict()
for pair in pairs:
    if pair[0][0] == pair[0][1]:
        continue
    if pair[0][0] not in repl_dict:
        repl_dict[pair[0][0]] = []
    repl_dict[pair[0][0]].append([pair[0][1], pair[1]])

all_letters = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'

In [70]:
repl_dict

{'о': [['а', 131], ['и', 6], ['у', 6], ['е', 5], ['ы', 3]],
 'ы': [['и', 122], ['о', 6], ['е', 4]],
 'и': [['е', 113], ['ы', 32], ['й', 31], ['о', 6], ['у', 4]],
 'а': [['о', 88], ['и', 16], ['е', 13], ['я', 11]],
 'е': [['и', 72], ['о', 23], ['э', 14], ['я', 9], ['а', 3], ['й', 3]],
 'щ': [['ш', 45], ['ж', 3]],
 'з': [['с', 42]],
 'с': [['з', 38], ['ц', 5]],
 'э': [['е', 33]],
 'й': [['и', 26]],
 'я': [['е', 19], ['а', 18], ['и', 14]],
 'д': [['т', 13]],
 'ъ': [['ь', 9]],
 'ж': [['ш', 8], ['з', 3]],
 'ш': [['щ', 8], ['с', 6], ['ж', 5]],
 'т': [['д', 8], ['ц', 3]],
 'ч': [['ш', 7], ['щ', 4]],
 'ь': [['ъ', 6], ['и', 5]],
 'г': [['в', 6], ['к', 4]],
 'л': [['д', 6]],
 'ц': [['ч', 6]],
 'к': [['г', 5]],
 'р': [['п', 4]],
 'у': [['о', 4], ['ы', 3]],
 'в': [['б', 3]],
 'н': [['х', 3]],
 'б': [['п', 3]],
 'ю': [['и', 2]]}

In [71]:
def replace_letter(word, subset=all_letters, prob=0.2):
    new_word = ''
    
    for l in word:
        if l in repl_dict and l in subset:
            r = choice([True, False], 1, p=[prob, 1 - prob]) # выбираем, совершать ли замену
            if r:
                letters = [l for l, n in repl_dict[l]] # список возможных букв
                l_sum = sum(n for _, n in repl_dict[l])
                distr = [n/l_sum for l, n in repl_dict[l]] # распределение вероятностей, на некоторые заменяют чаще, на некоторые реже
                new_letter = choice(letters, 1, p=distr)[0]
                new_word += new_letter
            else:
                new_word += l
        else:
            new_word += l
    return new_word

In [79]:
replace_letter('телевизор')

'толебизор'

In [84]:
replace_letter('выходные')

'быходные'

### 2. Убирание буквы из кластера согласных

In [ ]:
# ищем кластеры регуляркой, про каждый понимаем его длину и с некоторой вероятностью удаляем какую-то из букв

In [86]:
cl_pattern = '[бвгджзйклмнпрстфхцчшщ]{2,}'
cl_distr = [70/(70+42+63), 42/(70+42+63), 63/(70+42+63)]
cl_distr_2 = [70/(70+63), 63/(70+63)]

def reduce_cluster(word, prob=0.5):
    new_word = word
    for m in re.finditer(cl_pattern, word):
        r = choice([True, False], 1, p=[prob, 1 - prob])
        if r:
            if m.end(0) - m.start(0) > 2:
                c_len = m.end(0) - m.start(0) - 2
                new_distr = [cl_distr[0]]+[cl_distr[1]/c_len]*c_len+[cl_distr[2]]
                l_i = choice(list(range(0, m.end(0) - m.start(0))), 1, p=new_distr)[0]
            else:
                l_i = choice([0, 1], 1, p=cl_distr_2)[0]
            new_word = new_word[:m.start(0)+l_i] + ' ' + new_word[m.start(0)+l_i+1:]
    return new_word.replace(' ', '')

In [145]:
reduce_cluster('приветствовать')

'приветстовать'

In [98]:
reduce_cluster('детства')

'детства'

### 3. Мягкий знак

In [160]:
del_b_pattern = 'ь'

def del_b(word, prob=0.5):
    new_word = word
    for m in re.finditer(del_b_pattern, word):
        r = choice([True, False], 1, p=[prob, 1 - prob])
        if r:
            new_word = new_word[:m.start(0)] + ' ' + new_word[m.start(0)+1:]
    return new_word.replace(' ', '') 

In [174]:
del_b('уменьшительность')

'уменшительност'

In [213]:
add_b_pattern = '[бвгджзйклмнпрстфхцчшщ]{2}[еиюяь]'

def add_b(word, prob=0.5):
    s = 0
    bs_parts = []
    new_word = word
    for m in re.finditer(add_b_pattern, word):
        r = choice([True, False], 1, p=[prob, 1 - prob])
        if r:
            new_word = new_word[:m.start(0)+s+1] + 'ь' + new_word[m.start(0)+s+1:]
            s += 1
    if new_word[-1] not in 'аеиоуюяыэьйъ':
        r = choice([True, False], 1, p=[prob, 1 - prob])
        if r:
            new_word += 'ь'
    
    return new_word

In [216]:
add_b('приветриват')

'приветьривать'

### 4. Дефисное/пробельное/слитное написание

In [101]:
h_s_pattern = '[\- ]'
hyph_distr = [21/(136+21), 136/(136+21)]

def hyphen_space(word, prob = 0.5):
    new_word = word
    if '-' in word:
        distr = [prob*hyph_distr[0], 1 - prob, prob*hyph_distr[1]]
        r = choice(['', '-', ' '], 1, p=distr)[0]
        new_word = new_word.replace('-', r)   
    else:
        r = choice(['', '-', ' '], 1, p=[1/3, 1/3, 1/3])[0]
        new_word = new_word.replace(' ', r)    
    return new_word

In [115]:
hyphen_space('по-английски')

'по английски'

### 5. Редукция и гиперкоррекция

In [230]:
def get_aot_json(word):
    url = 'http://63.250.59.227:8080/?'
    data = {'action': 'morph',
            'langua': 'Russian',
            'query': word,
            'withparadigms': '1',}
    html_content = requests.get(url, data).text
    return html_content

In [231]:
def get_wordlist(word):
    a = get_aot_json(word)
    b = json.loads(a)
    forms_list = []
    for paradigm in b[0]['paradigm']:
        for formsGroup in paradigm['formsGroups']:
            for form in formsGroup['forms']:
                forms_list.append(form['f'].lower())
    return forms_list

In [272]:
red_dict = dict()

red_dict['е'] = 'и'
red_dict['о'] = 'а'
red_dict['я'] = 'и'
red_dict['э'] = 'ы'

In [256]:
def reduction(word, prob = 0.5):
    wordlist = get_wordlist(word)
    wordlist_unaccent = [word.replace("'", '') for word in wordlist]
    try:
        in_list = wordlist_unaccent.index(word)
        my_word = wordlist[in_list]
    except:
        print('Слово не нашлось в словаре!')
        return word
    if "'" not in my_word:
        print('Слово не нашлось в словаре!')
        return word
    for i in range(len(word)):
        if my_word[i] in 'еояэ' and my_word[i+1] != "'":
            r = choice([True, False], 1, p=[prob, 1 - prob])[0]
            if r:
                my_word = my_word[:i] + red_dict[my_word[i]] + my_word[i+1:]
    return my_word.replace("'", '')

In [265]:
reduction('одеяло', prob=1)

'адияло'

In [273]:
un_red_dict = dict()

un_red_dict['а'] = 'о'
un_red_dict['и'] = 'е'

In [274]:
def un_reduction(word, prob = 0.5):
    wordlist = get_wordlist(word)
    wordlist_unaccent = [word.replace("'", '') for word in wordlist]
    try:
        in_list = wordlist_unaccent.index(word)
        my_word = wordlist[in_list]
    except:
        print('Слово не нашлось в словаре!')
        return word
    if "'" not in my_word:
        print('Слово не нашлось в словаре!')
        return word
    for i in range(len(word)):
        if my_word[i] in 'иа' and my_word[i+1] != "'":
            r = choice([True, False], 1, p=[prob, 1 - prob])[0]
            if r:
                my_word = my_word[:i] + un_red_dict[my_word[i]] + my_word[i+1:]
    return my_word.replace("'", '')

In [275]:
un_reduction('автострада', prob=1)

'овтострада'

## Морфологические

1. Поиск похожего (с зафиксированным префиксом)
2. Изменение/удаление префикса
3. Изменение окончания (парадигмы)
4. Чередующиеся основы


### 1. Поиск похожего (изменение/удаление префикса)

In [48]:
prefs = 'в- (во-), взо-, вне-, внутри-, возо-, вы-, до-, еже-, за-, зако-, изо-, испод-, к-, кое-, ку-, меж- (междо-, между-), на-, над- (надо-), наи-, не-, недо-, ни-, низо-, о-, об- (обо-), около-, от- (ото-), па-, пере-, по-, под- (подо-), поза-, после-, пра-, пред- (преди-, предо-), про-, противо-, разо-, с- (со-), сверх-, среди-, су-, тре-, у-, пре-, при-.'[:-1].replace('-', '').replace(' (', ', ').replace(')', '').split(', ')
prefs.extend('без-/бес-, вз-/вс-, воз-/вос-, из-/ис-, низ-/нис-, обез-/обес-, раз-/рас-, роз-/рос-, через-/черес- (чрез-/чрес-)'.replace('-', '').replace('/', ', ').replace(' (', ', ').replace(')', '').split(', '))

In [49]:
freq_verbs_df = pd.read_csv('freq_verbs.csv', encoding='utf-8')
freq_verbs_list = freq_verbs_df['Лемма'].tolist()

In [50]:
def find_pref(word, prefs):
    max_pref = ''
    len_pref = 0

    for i in range(len(word)):
        if word[:(i+1)] in prefs:
            max_pref = word[:(i+1)]
            len_pref = i+1
    
    return max_pref

In [55]:
def modify_pref(cor_verb):

    cor_pref = find_pref(cor_verb, prefs)
    stem = morph.parse(cor_verb)[0].normal_form[len(cor_pref):]

    fr_verb = None
    for verb in freq_verbs_list:
        if verb.endswith(stem):
            fr_verb = verb
            break
    if fr_verb:
        form = fr_verb[:-len(stem)]+cor_verb[len(cor_pref):]
        return form
    else:
        form = None
        print('Не нашлось :(')
        return cor_verb
    

In [60]:
modify_pref('недоделать')

'сделать'

### 2. Поиск похожего (с зафиксированным префиксом)

In [2]:
with open('russian.txt', 'r') as f:
    words = f.read().split('\n')

In [35]:
def simil_pref(my_word):
    pre = my_word[:2]
    pre_words = [word for word in words if word.startswith(pre)]
    pre_dist = [(word, nltk.edit_distance(word, my_word)) for word in pre_words]
    
    for word in sorted(pre_dist, key=lambda x: x[1])[1:]:
        if morph.parse(my_word)[0].tag == morph.parse(word[0])[0].tag:
            return word[0]
    print('Ничего не нашлось :(')
    return my_word

In [36]:
simil_pref('собака')

'собачка'

### 3. Изменение окончания (парадигмы)

In [37]:
flex_dict = dict()
flex_dict['ед'] = dict()
flex_dict['мн'] = dict()
flex_dict['ед']['им'] = ['а', 'я', 'о', 'е', '', 'ь', 'й']
flex_dict['ед']['рд'] = ['и', 'ы', 'а', 'я']
flex_dict['ед']['дт'] = ['и', 'е', 'у', 'ю']
flex_dict['ед']['вн'] = ['у', 'ю', 'ь', '', 'й', 'а', 'я']
flex_dict['ед']['тв'] = ['ой', 'ей', 'ью', 'ом', 'ем']
flex_dict['ед']['пр'] = ['е', 'и']

flex_dict['мн']['им'] = ['а', 'я', 'ы', 'и']
flex_dict['мн']['рд'] = ['', 'ь', 'ей', 'ов', 'ев', 'й']
flex_dict['мн']['дт'] = ['ам', 'ям']
flex_dict['мн']['вн'] = ['й', 'ев', 'ь', 'ов', 'ей', 'а', 'я', 'ы', 'и']
flex_dict['мн']['тв'] = ['ами', 'ями']
flex_dict['мн']['пр'] = ['ах', 'ях']

vowels = ['а','е','и','о','у','ю','я','ы','э']
hard_cons = ['ц', 'ш', 'ж', 'ч', 'щ', 'к', 'г']

In [38]:
def find_flex(word, flexes):
    max_flex = ''
    len_flex = 0

    for i in range(len(word)):
        if word[-(i+1):] in flexes:
            max_flex = word[-(i+1):]
            len_flex = i+1
    
    return max_flex

In [39]:
def change_paradigm(word, case=None, number=None):
    
    if not case and not number:
        ana = morph.parse(word)[0]
        case = morph.lat2cyr(ana.tag.case)
        number = morph.lat2cyr(ana.tag.number)
    
    elif not case:
        anas = morph.parse(word)
        for ana in anas:
            if morph.lat2cyr(ana.tag.number) == number:
                case = morph.lat2cyr(ana.tag.case)
                break
    
    elif not number:
        anas = morph.parse(word)
        for ana in anas:
            if morph.lat2cyr(ana.tag.case) == case:
                number = morph.lat2cyr(ana.tag.number)
                break
            
    print(ana)
    flexes = copy(flex_dict[number][case])
    
    flex = find_flex(word, flexes)
    
    stem = word[:-len(flex)]
    if stem[-1] in vowels:
        try:
            flexes.remove('ь')
        except ValueError:
            pass
        try:
            flexes.remove('')
        except ValueError:
            pass
    else:
        try:
            flexes.remove('й')
        except ValueError:
            pass
        if stem[-1] in hard_cons:
            try:
                flexes.remove('ю')
            except ValueError:
                pass
            try:
                flexes.remove('я')
            except ValueError:
                pass
            try:
                flexes.remove('ю')
            except ValueError:
                pass
    
    flexes.remove(flex)
    new_flex = random.choice(flexes)
    new_word = stem+new_flex
    
    return new_word

In [40]:
change_paradigm('родители', case=None)

Parse(word='родители', tag=OpencorporaTag('NOUN,anim,masc plur,nomn'), normal_form='родитель', score=1.0, methods_stack=((DictionaryAnalyzer(), 'родители', 122, 6),))


'родителы'

### 4. Чередующиеся основы

In [124]:
lemma_flexes = copy(flex_dict['ед']['им'])
alt_pattern = r'([цшжчщкг])я'

def altern(word):
    ana = morph.parse(word)[0]
    pos = ana.tag.POS
    if pos == 'NOUN':
        
        case = morph.lat2cyr(ana.tag.case)
        number = morph.lat2cyr(ana.tag.number)
        flexes = copy(flex_dict[number][case])
        
        flex = find_flex(word, flexes)
        
        lemma = ana.normal_form
        lemma_flex = find_flex(lemma, lemma_flexes)
        
        if len(lemma_flex) == 0:
            new_word = lemma + flex
        else:
            new_word = lemma[:-len(lemma_flex)] + flex
        
        new_word = re.sub(alt_pattern, r'\1а', new_word)
        return new_word

In [126]:
altern('прогулок')

'прогулк'